In [1]:
from src.aaa_image_enhancement.defects_detection_fns import is_low_light, is_dark_color, is_dark_histogram, is_dark_threshold, is_dark_local_contrast, is_dark_blocks, is_dark_edges, is_dark_adaptive_threshold, is_dark_v_channel
from src.aaa_image_enhancement.image_utils import ImageConversions
from src.aaa_image_enhancement.image_defects_detection import DefectNames
from collections import Counter
from PIL import Image
from typing import List, Dict
import glob
import time
from IPython.display import display, clear_output

In [2]:
def defect_name_to_string(defect: DefectNames) -> str:
    """Convert DefectNames enum to a human-readable string."""
    return defect.name.replace('_', ' ').capitalize()

def test_detection(image_paths: List[str]) -> Dict[str, Dict[str, List[str]]]:
    result = {}
    timings = {}
    
    for image_path in image_paths:
        image = Image.open(image_path)
        image_conv = ImageConversions(image)
        
        # Dictionary to store timing for each method
        method_timings = {}
        
        # Run the detection methods and time them
        start_time = time.time()
        result_low_light = is_low_light(image_conv, threshold=125)
        method_timings['is_low_light'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_color = is_dark_color(image_conv, threshold=150)
        method_timings['is_dark_color'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_histogram = is_dark_histogram(image_conv)
        method_timings['is_dark_histogram'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_threshold = is_dark_threshold(image_conv)
        method_timings['is_dark_threshold'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_local_contrast = is_dark_local_contrast(image_conv)
        method_timings['is_dark_local_contrast'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_adaptive_threshold = is_dark_adaptive_threshold(image_conv)
        method_timings['is_dark_adaptive_threshold'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_v_channel = is_dark_v_channel(image_conv)
        method_timings['is_dark_v_channel'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_edges = is_dark_edges(image_conv)
        method_timings['is_dark_edges'] = time.time() - start_time
        
        start_time = time.time()
        result_dark_blocks = is_dark_blocks(image_conv)
        method_timings['is_dark_blocks'] = time.time() - start_time
        
        # Combine all results into one dictionary
        combined_results = {
            **result_low_light,
            **result_dark_color,
            **result_dark_histogram,
            **result_dark_threshold,
            **result_dark_local_contrast,
            **result_dark_adaptive_threshold,
            **result_dark_v_channel,
            **result_dark_edges,
            **result_dark_blocks,
        }
        
        detected_defects = [
            defect_name_to_string(defect) for defect, detected in combined_results.items() if detected
        ]
        
        result[image_path] = detected_defects
        timings[image_path] = method_timings
        
    return {'results': result, 'timings': timings}

def analyze_detection_results(detection_results: Dict[str, List[str]]) -> Dict[str, int]:
    """Analyze the detection results to find the most common defects.

    Args:
        detection_results: A dictionary where keys are image paths and values are lists of detected defects.

    Returns:
        A dictionary with defect names as keys and their occurrence counts as values.
    """
    all_defects = []
    for defects in detection_results.values():
        all_defects.extend(defects)

    defect_counts = Counter(all_defects)
    
    return dict(defect_counts)

def calculate_average_timings(timings: Dict[str, Dict[str, float]]) -> Dict[str, float]:
    """Calculate the average execution time for each detection method.

    Args:
        timings: A dictionary where keys are image paths and values are dictionaries with method timings.

    Returns:
        A dictionary with method names as keys and their average execution times as values.
    """
    total_times = Counter()
    counts = Counter()
    
    for method_timings in timings.values():
        for method, timing in method_timings.items():
            total_times[method] += timing
            counts[method] += 1
    
    average_times = {method: total_times[method] / counts[method] for method in total_times}
    
    return average_times

# Get image paths
dark_image_paths = glob.glob("../data/low-light-validation/*.jpg")
light_image_paths = glob.glob("../data/high-light-validation/*.jpg")

# Get detection results and timings
low_light_res_with_timings = test_detection(dark_image_paths)
high_light_res_with_timings = test_detection(light_image_paths)

# Extract the actual detection results
low_light_res = low_light_res_with_timings['results']
high_light_res = high_light_res_with_timings['results']

# Analyze detection results
low_light_analysis = analyze_detection_results(low_light_res)
high_light_analysis = analyze_detection_results(high_light_res)

# Output the timings for analysis
low_light_timings = low_light_res_with_timings['timings']
high_light_timings = high_light_res_with_timings['timings']

# Calculate average timings
average_low_light_timings = calculate_average_timings(low_light_timings)
average_high_light_timings = calculate_average_timings(high_light_timings)

print("Low Light Analysis Results:", low_light_analysis)
print("High Light Analysis Results:", high_light_analysis)
print("Average Low Light Timings:", average_low_light_timings)
print("Average High Light Timings:", average_high_light_timings)

Low Light Analysis Results: {'Low light': 106, 'Dark light': 86, 'Dark histogram': 7, 'Dark threshold': 7, 'Dark local contrast': 7, 'Dark v channel': 4, 'Dark blocks': 8}
High Light Analysis Results: {'Low light': 2, 'Dark light': 3}
Average Low Light Timings: {'is_low_light': 0.0005613247553507487, 'is_dark_color': 0.014052939414978028, 'is_dark_histogram': 0.00565109650293986, 'is_dark_threshold': 0.00037953058878580727, 'is_dark_local_contrast': 0.0009136617183685303, 'is_dark_adaptive_threshold': 0.002025318145751953, 'is_dark_v_channel': 0.0009271899859110514, 'is_dark_edges': 0.0012202819188435873, 'is_dark_blocks': 0.0028901636600494386}
Average High Light Timings: {'is_low_light': 0.0007022937138875325, 'is_dark_color': 0.020223371187845864, 'is_dark_histogram': 0.007092881202697754, 'is_dark_threshold': 0.000465846061706543, 'is_dark_local_contrast': 0.0011879205703735352, 'is_dark_adaptive_threshold': 0.002680365244547526, 'is_dark_v_channel': 0.0011633714040120444, 'is_dark

In [3]:
low_light_analysis

{'Low light': 106,
 'Dark light': 86,
 'Dark histogram': 7,
 'Dark threshold': 7,
 'Dark local contrast': 7,
 'Dark v channel': 4,
 'Dark blocks': 8}

In [4]:
high_light_analysis

{'Low light': 2, 'Dark light': 3}